# 정형화 Weight Regularization 

- 최적화 함수의 weight_decay 로 강도를 조절할 수 있습니다.
- ex) torch.optim.SGD(params, lr=1, momentum=0, dampening=0, weight_decay=0, nesterov=False)
- 모델이 오버피팅할 경우, 적절한 강도로 정형화를 걸어주면 이를 어느정도 극복할 수 있습니다.
- 정형화 부분 빼고는 컨볼루션 인공신경망 코드와 동일합니다.

In [ ]:
# 런타임 유형을 GPU로 바꾸시길 추천드립니다.
# !pip install torch torchvision

## 1. Settings
### 1) Import required libraries

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 2) Set hyperparameters

In [ ]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [ ]:
mnist_train = dset.MNIST('./',
                         train = True,
                         download = True,
                         transform = transforms.ToTensor(),
                         target_transform = None)
mnist_test = dset.MNIST('./',
                        train = False,
                        download = True,
                        transform = transforms.ToTensor(),
                        target_transform = None)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 2) Check Dataset

In [ ]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


### 3) Set DataLoader

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_train,
                                           batch_size = batch_size,
                                           shuffle = True,
                                           num_workers = 2,
                                           drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          num_workers = 2,
                                          drop_last = True)

## 3. Model & Optimizer

### 1) CNN Model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1),    # 28 x 28
            nn.ReLU(),
            nn.Conv2d(16, 32, 3,padding = 1),    # 28 x 28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                  # 14 x 14
            nn.Conv2d(32, 64, 3, padding = 1),   # 14 x 14
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                   # 7 x 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )       
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

### 2) Loss func & Optimizer

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
# Regularization(규제)은 weight_decay 인수로 줄 수 있습니다.
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay = 0.1) # 람다(lambda)값이 0.1인 L2 규제 설정

cuda:0


## 4. Train 

In [ ]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 0:
        print(loss)          

tensor(2.3020, device='cuda:0', grad_fn=<NllLossBackward>)


In [ ]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[[[-0.1850,  0.1515, -0.2976],
          [ 0.2102,  0.2367, -0.0527],
          [-0.2006,  0.0757,  0.2432]]],


        [[[-0.1046,  0.0936,  0.0208],
          [ 0.1926,  0.3115,  0.2037],
          [ 0.2889,  0.2824,  0.0662]]],


        [[[-0.2835,  0.0801,  0.0056],
          [-0.2819,  0.1096,  0.2440],
          [ 0.1505,  0.2453, -0.0731]]],


        [[[ 0.0891,  0.2647, -0.0605],
          [ 0.1230, -0.3116,  0.1068],
          [-0.0334,  0.1669, -0.0474]]],


        [[[-0.1074,  0.2479, -0.2726],
          [-0.1133, -0.2822,  0.0978],
          [-0.1871, -0.1557,  0.0715]]],


        [[[ 0.1196,  0.0896, -0.0388],
          [-0.2996, -0.2430, -0.2807],
          [ 0.0181, -0.0267,  0.2566]]],


        [[[-0.2741,  0.1240, -0.2626],
          [-0.0699,  0.0241,  0.1578],
          [ 0.2697, -0.0884, -0.0374]]],


        [[[-0.2154,  0.3133, -0.3149],
          [ 0.2996,  0.1955, -0.1396],
          [ 0.2431, -0.1104, -0.2741]]],


        [

## 5. Test

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for image, label in test_loader:
      x = image.to(device)
      y_ = label.to(device)

      output = model.forward(x)
      _, output_index = torch.max(output, 1)  # torch.max()는 최댓값이 들어있는 index를 리턴함

      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100 * correct / total))

Accuracy of Test Data: 12.079326629638672


## 정확도가 왜 이렇게 낮을까?

- 오버피팅(과적합)하지 않는 상태에서 Regularization을 사용하면 오히려 학습이 잘 안됩니다.